In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import logging
import pandas as pd
import numpy as np
import yaml

sys.path.append(os.path.dirname(os.path.abspath('')))
#from utils.spark import restart_spark
from auto_ab.stratification.params import SplitBuilderParams
from auto_ab.prepilot.params import PrepilotParams
from auto_ab.auto_ab.abtest import ABTest
from auto_ab.auto_ab.params import ABTestParams

logging.basicConfig(level = logging.INFO)

%load_ext autoreload
%autoreload 2
from auto_ab.stratification.split_builder import StratificationSplitBuilder

# Кейс №6. Выделение групп

In [2]:
df = pd.read_csv('ab_data.csv')
df["moda_city"] = np.random.randint(1, 5, df.shape[0])
df["moda_city"] = df["moda_city"].astype(str)
df["country"] = np.random.randint(1, 3, df.shape[0])
df["id"] = df.index

In [3]:
split_builder_params = SplitBuilderParams(
    map_group_names_to_sizes={
        'control': 10000,
        'target': 10000,
        'target1': 10000
    },
    region_col = "moda_city",
    split_metric_col = "height_now",
    customer_col = "id",
    cols = [],
    cat_cols=["country"
        
    ],
    pvalue=0.05,
    n_top_cat=100,
    stat_test="ttest_ind"
)

In [4]:
split_builder = StratificationSplitBuilder(df, split_builder_params)

In [5]:
guests_data_with_strata = split_builder.assign_strata()

INFO:auto_ab.stratification.split_builder:Calculate stratas for guest table
INFO:auto_ab.stratification.binning:20418 outliers found


In [6]:
guests_data_with_strata.head()

,height_now,height_prev,weight_now,weight_prev,noise_now,noise_prev,groups,id,moda_city,country,numerator,denominator,conversion,strata
0,172.470442,166.382868,163.648256,173.422115,5.644384,-1.923929,A,0,2,0.50052,3,2,1,201
8,171.875892,170.127211,164.898452,177.149415,5.975044,1.970891,B,8,2,0.49948,3,4,1,201
9,171.061403,166.374731,167.835024,172.793484,4.044180,7.901575,B,9,2,0.50052,2,2,1,201
12,172.760235,168.417151,165.007237,175.096232,8.241634,-0.112896,B,12,2,0.50052,1,4,1,201
17,167.449021,163.500754,161.494712,169.214959,10.096040,6.560057,A,17,2,0.50052,2,4,1,201


In [7]:
groups = split_builder.build_split(guests_data_with_strata)

INFO:auto_ab.stratification.split_builder:control: Desired size = 0.05,             resulting size = 10000, diff = 0.0 %
INFO:auto_ab.stratification.split_builder:target: Desired size = 0.05263157894736842,             resulting size = 10002, diff = 0.02 %
INFO:auto_ab.stratification.split_builder:target1: Desired size = 0.05555617284636496,             resulting size = 10013, diff = 0.13 %
INFO:auto_ab.stratification.split_builder:Success!


In [8]:
groups

,height_now,height_prev,weight_now,weight_prev,noise_now,noise_prev,groups,id,moda_city,country,numerator,denominator,conversion,strata,group_name
12,172.760235,168.417151,165.007237,175.096232,8.241634,-0.112896,B,12,2,0.50052,1,4,1,201,control
17,167.449021,163.500754,161.494712,169.214959,10.096040,6.560057,A,17,2,0.50052,2,4,1,201,control
21,168.746442,164.330298,166.211897,170.709041,5.782766,5.921877,A,21,2,0.49948,2,3,1,201,target
46,171.216888,168.034656,164.780729,173.399175,7.656002,5.082804,A,46,2,0.49948,2,4,1,201,target
231,170.697699,168.341729,166.961396,172.793921,8.810225,3.338596,A,231,2,0.49948,3,3,1,201,control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199594,176.915536,177.405746,173.892404,179.771035,5.017932,8.363306,B,199594,1,0.49948,4,3,1,110,target1
199787,174.957342,174.103153,169.466644,177.946197,6.669149,1.321797,A,199787,1,0.49948,1,4,1,111,target
199819,173.299608,169.234741,166.165019,175.920923,-1.187242,4.051627,A,199819,1,0.50052,3,4,0,11-1,control
199850,174.099399,173.439629,171.903040,174.862695,7.531154,9.796080,B,199850,1,0.50052,3,2,1,111,control
